In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from matplotlib import pyplot as plt 
import cv2 as cv

# Introduction

In this notebook I will be using YOLO V7 for object detection. This notebook will be different from most of my other notebooks in that I will be cloning a github repo and working with the kaggle filesystem to make this functional.

I am using YoloV7 from https://github.com/WongKinYiu/yolov7 There are actually multiple claims to YOLO Version 7 due to seven being a lucky number. I have chosen this one principally because I find it the most user friendly of the claimants.

The repo I have chosen is an implementation of https://arxiv.org/abs/2207.02696

This notebook is very similar to my previous notebook with YOLO version 5 at: https://www.kaggle.com/code/taranmarley/yolo-v5-object-detection/
I am not entirely trying to be original though I have made some iterative improvements to the notebook beyond using a new version of YOLO

# Look at Some Images

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(10,5))
image = cv.imread("../input/sptire/train/images/14_19_l_jpg.rf.8323d9f848377e32ca451017a3a80731.jpg")
ax[0].imshow(image)
image = cv.imread("../input/sptire/train/images/IMG_0719_JPEG.rf.05f197445c4a42854e0b1f308fb4e636.jpg")
ax[1].imshow(image)
image = cv.imread("../input/sptire/train/images/IMG_0680_JPEG.rf.560c49e01182db8356989ddc604557fb.jpg")
ax[2].imshow(image)
image = cv.imread("../input/sptire/train/images/IMG_0701_JPEG.rf.d5ae66ab383142ef5d59b0454a19fdce.jpg")
ax[3].imshow(image)
fig.show()

# Create the File System

In [ ]:
%cd ../
!mkdir tmp
%cd tmp

# Download YoloV7

In [ ]:
# Download YOLOv7
!git clone https://github.com/WongKinYiu/yolov7 # clone repo
%cd yolov7
# Install dependencies
%pip install -qr requirements.txt  # install dependencies

%cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# New YAML file

Create new Yaml file to fit the file system. 

In [ ]:
%cp -r ../input/sptire /kaggle/tmp

In [ ]:
# Create .yaml file 
import yaml

data_yaml = dict(
    train = '../sptire/train',
    val = '../sptire/valid',
    nc = 1,
    names = ['Tire']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [ ]:
%cd yolov7

# Train the YoloV7 Model

In [ ]:
!wandb disabled
!python train.py --img 416 --batch 16 --epochs 50 --data ../data.yaml  --weights 'yolov7.pt'

# Look at Training Results

In [ ]:
img = cv.imread("runs/train/exp/results.png")
plt.figure(figsize=(15, 15))
plt.imshow(img)

# Model Validation Output

Let's look at how the model performed on a validation batch

In [ ]:
img = cv.imread("runs/train/exp/test_batch2_pred.jpg")
plt.figure(figsize=(15, 15))
plt.imshow(img)

# Test the model

This will test the model we have created against an image it has not seen previously.

In [ ]:
!python detect.py --source ../../input/sptire/test/images/IMG_0672_JPEG.rf.c37833de9c2310cfba797a83f239d3c1.jpg --weights runs/train/exp/weights/best.pt

In [ ]:
img = cv.imread("runs/detect/exp/IMG_0672_JPEG.rf.c37833de9c2310cfba797a83f239d3c1.jpg")
plt.imshow(img)

# Conclusion

The results from this weren't better than my previous attempts with YOLO V5. This however isn't a scientific comparison of the two and any important project would be wise to use a variety of available approaches until they find the best one for that specific use case. 

A real comparison of the techniques would need to take into account differences in parameter size, speed, memory usage, accuracy and a variety of further factors over many runs to determine which is working better with the dataset. 

This however is just an exploration of YOLO V7 and I commend that they made it user friendly by providing documentation, keeping it similar to V5 and Google notebooks of it being used. 